In [180]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [181]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [182]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [183]:
# create engine to hawaii.sqlite
database_path = "hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
# conn = engine.connect()

In [184]:
# reflect an existing database into a new model
measurement_data = engine.execute("SELECT * FROM Measurement")
# # reflect the tables
for record in measurement_data:
    print(record)

.38, 70.0)
(19140, 'USC00516128', '2016-06-16', 0.12, 70.0)
(19141, 'USC00516128', '2016-06-17', 1.27, 69.0)
(19142, 'USC00516128', '2016-06-18', 0.72, 73.0)
(19143, 'USC00516128', '2016-06-19', 0.79, 74.0)
(19144, 'USC00516128', '2016-06-20', 0.52, 72.0)
(19145, 'USC00516128', '2016-06-21', 0.78, 71.0)
(19146, 'USC00516128', '2016-06-22', 0.2, 73.0)
(19147, 'USC00516128', '2016-06-23', 0.9, 72.0)
(19148, 'USC00516128', '2016-06-25', None, 73.0)
(19149, 'USC00516128', '2016-06-26', 0.7, 74.0)
(19150, 'USC00516128', '2016-06-27', 0.3, 72.0)
(19151, 'USC00516128', '2016-06-28', 0.25, 74.0)
(19152, 'USC00516128', '2016-06-29', 0.35, 72.0)
(19153, 'USC00516128', '2016-07-01', None, 74.0)
(19154, 'USC00516128', '2016-07-04', None, 74.0)
(19155, 'USC00516128', '2016-07-05', 0.45, 71.0)
(19156, 'USC00516128', '2016-07-06', 1.46, 71.0)
(19157, 'USC00516128', '2016-07-07', 0.92, 73.0)
(19158, 'USC00516128', '2016-07-08', 0.31, 73.0)
(19159, 'USC00516128', '2016-07-09', 0.15, 71.0)
(19160, 'USC0

In [185]:
# reflect an existing database into a new model
station_data = engine.execute("SELECT * FROM Station")
# # reflect the tables
for record in station_data:
    print(record)

(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)
(2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6)
(3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0)
(4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9)
(5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)
(6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5)
(7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9)
(8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9)
(9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)


In [186]:
# View all of the classes that automap found
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [187]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [188]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [189]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [190]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 


# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
previous_year = dt.datetime(2017,8,23)-dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= previous_year).all()
#print(results)

# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




 ('2016-12-13', 0.15), ('2016-12-14', 0.05), ('2016-12-15', 0.0), ('2016-12-16', 0.0), ('2016-12-17', 0.16), ('2016-12-18', 0.27), ('2016-12-19', 0.02), ('2016-12-20', 0.01), ('2016-12-21', 0.06), ('2016-12-22', 0.14), ('2016-12-23', 0.02), ('2016-12-24', 0.06), ('2016-12-25', 0.0), ('2016-12-26', 0.06), ('2016-12-27', 0.0), ('2016-12-28', 0.06), ('2016-12-29', 0.05), ('2016-12-30', 0.07), ('2017-01-01', None), ('2017-01-03', None), ('2017-01-04', 0.18), ('2017-01-05', 0.42), ('2017-01-06', 0.01), ('2017-01-07', 0.0), ('2017-01-08', 0.0), ('2017-01-09', 0.0), ('2017-01-10', 0.0), ('2017-01-12', None), ('2017-01-15', None), ('2017-01-16', 0.0), ('2017-01-18', None), ('2017-01-19', 0.0), ('2017-01-20', 0.0), ('2017-01-21', 0.11), ('2017-01-22', 0.04), ('2017-01-23', 0.0), ('2017-01-24', 0.08), ('2017-01-25', 0.0), ('2017-01-26', 0.0), ('2017-01-27', 0.0), ('2017-01-29', None), ('2017-01-30', 0.0), ('2017-01-31', 0.0), ('2017-02-01', 0.0), ('2017-02-02', 0.0), ('2017-02-03', 0.0), ('2017-

In [191]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [192]:
# Design a query to calculate the total number stations in the dataset


In [193]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [194]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [195]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [196]:
# Close Session
session.close()